# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [16]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make -j install

mkdir: cannot create directory ‘build’: File exists
-- The C compiler identification is GNU 9.2.0
-- The CXX compiler identification is GNU 9.2.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
Habilitando Log de Funciones de nivel 1
Release mode
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found OpenMP: TRUE (found version "4.5")  
-- Found libomp without any special flags
-- Found PythonInterp: /home/j/Code/mn-tp2/.venv/bin/python (found version "3.7.4") 
-- Found PythonLibs: /us

In [17]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment

/home/j/Code/mn-tp2/notebooks
Python 3.7.4


In [18]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz
!cd ../data && tar -xvf *.tar.gz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
tar: *.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
imdb_small.csv
Cantidad de documentos: 12500


In [19]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,This show comes up with interesting locations ...,neg,11644_10.txt
freq,6275,2,6322,2


In [20]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]

text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))

Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [21]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [22]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)

vectorizer.fit(text_train)

X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values

In [23]:
%%time
import sentiment
clf = sentiment.KNNClassifier(100)
clf.fit(X_train, y_train)

CPU times: user 662 ms, sys: 89 ms, total: 751 ms
Wall time: 754 ms


In [24]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.6353784860557768
CPU times: user 1min 13s, sys: 405 ms, total: 1min 14s
Wall time: 1min 14s


In [25]:
%%time
# lo mismo pero usando el clasificador de sklearn, a ver como andamos...
from sklearn.neighbors import KNeighborsClassifier
skl_clf = KNeighborsClassifier(100)
skl_clf.fit(X_train, y_train)

CPU times: user 24 ms, sys: 6.63 ms, total: 30.6 ms
Wall time: 681 ms


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=100, p=2,
           weights='uniform')

In [27]:
%%time
from sklearn.metrics import accuracy_score
y_pred = skl_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.6387250996015936
CPU times: user 3.98 s, sys: 417 ms, total: 4.4 s
Wall time: 4.48 s


In [28]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('../src/python')
import searcher

ModuleNotFoundError: No module named 'searcher'

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn_problem = searcher.KNNHyperParameters(X_train, y_train, y_test, classifier_from="sklearn")


In [ ]:
from simpleai.search.local import hill_climbing
hill_climbing(knn_problem)